In [1]:
import cv2
import numpy as np
from typing import List, Tuple, Optional
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
import warnings
from scipy import signal
import os
import time
import random

In [2]:
IO_FILE_NAME = "a.mp4"

In [3]:
class CustomVideo:
    def __init__(self, video_path: str):
        """
        Initialize the CustomVideo class with a video file path.
        
        Args:
            video_path (str): Path to the video file (mp4 or avi format)
        """
        if not Path(video_path).exists():
            raise FileNotFoundError(f"Video file not found: {video_path}")
            
        self.video_path = video_path
        self.cap = cv2.VideoCapture(video_path)
        
        # Store video properties
        self.fps = int(self.cap.get(cv2.CAP_PROP_FPS))
        self.frame_count = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.frames: List[np.ndarray] = []
        
        # Read all frames
        self._read_frames()
        
    def _read_frames(self) -> None:
        """Read all frames from the video into memory."""
        while self.cap.isOpened():
            ret, frame = self.cap.read()
            if not ret:
                break
            self.frames.append(frame)
        self.cap.release()
        
    def get_video_info(self) -> dict:
        """Return video properties as a dictionary."""
        return {
            'fps': self.fps,
            'frame_count': self.frame_count,
            'width': self.width,
            'height': self.height,
            'duration': self.frame_count / self.fps
        }
    
    def swap_red_blue(self) -> None: # TODO: DELETE
        """Swap red and blue channels in all frames."""
        for i in range(len(self.frames)):
            self.frames[i] = cv2.cvtColor(self.frames[i], cv2.COLOR_BGR2RGB)
            
    def apply_grayscale(self) -> None: # TODO: DELETE
        """Convert all frames to grayscale."""
        for i in range(len(self.frames)):
            gray = cv2.cvtColor(self.frames[i], cv2.COLOR_BGR2GRAY)
            self.frames[i] = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
            
    def apply_gaussian_blur(self, kernel_size: Tuple[int, int] = (5, 5)) -> None: # TODO: DELETE
        """
        Apply Gaussian blur to all frames.
        
        Args:
            kernel_size (tuple): Kernel size for Gaussian blur (default: (5, 5))
        """
        for i in range(len(self.frames)):
            self.frames[i] = cv2.GaussianBlur(self.frames[i], kernel_size, 0)
            
    def adjust_brightness(self, alpha: float = 1.0, beta: int = 0) -> None:
        """
        Adjust brightness and contrast of all frames.
        
        Args:
            alpha (float): Contrast control
            beta (int): Brightness control 
        """
        for i in range(len(self.frames)):
            self.frames[i] = cv2.convertScaleAbs(self.frames[i], alpha=alpha, beta=beta)
            
    def rotate_video(self, angle: float) -> None: # TODO: USE or DELETE
        """
        Rotate all frames by the specified angle.
        
        Args:
            angle (float): Rotation angle in degrees
        """
        for i in range(len(self.frames)):
            height, width = self.frames[i].shape[:2]
            rotation_matrix = cv2.getRotationMatrix2D((width/2, height/2), angle, 1.0)
            self.frames[i] = cv2.warpAffine(self.frames[i], rotation_matrix, (width, height))
            
    def save_video(self, output_path: str, codec: str = 'mp4v') -> None:
        """
        Save the processed video to a file.
        
        Args:
            output_path (str): Path to save the output video
            codec (str): Four-character code of codec (default: 'mp4v')
        """
        if not self.frames:
            raise ValueError("No frames to save")
            
        fourcc = cv2.VideoWriter_fourcc(*codec)
        out = cv2.VideoWriter(output_path, fourcc, self.fps, (self.width, self.height))
        
        for frame in self.frames:
            out.write(frame)
            
        out.release()
        
    def get_frame(self, frame_number: int) -> Optional[np.ndarray]:
        """
        Get a specific frame from the video.
        
        Args:
            frame_number (int): Frame number to retrieve
            
        Returns:
            numpy.ndarray or None: The requested frame if available
        """
        if 0 <= frame_number < len(self.frames):
            return self.frames[frame_number]
        return None

    def apply_custom_effect(self, effect_func) -> None:
        """
        Apply a custom effect function to all frames.
        
        Args:
            effect_func (callable): Function that takes a frame and returns processed frame
        """
        for i in range(len(self.frames)):
            self.frames[i] = effect_func(self.frames[i])
            
    def make_extremely_bright(self, intensity: float = 3.0) -> None:
        """
        Make the video extremely bright using multiple techniques.
        
        Args:
            intensity (float): Brightness intensity multiplier (default: 3.0)
                             Higher values make the video even brighter
        """
        # First increase contrast and brightness significantly
        self.adjust_brightness(alpha=intensity, beta=100)
        
        # Then boost pixel values directly
        for i in range(len(self.frames)):
            # Convert to float for processing
            frame_float = self.frames[i].astype(float)
            
            # Boost pixel values
            frame_float = frame_float * intensity
            
            # Add brightness offset
            frame_float = frame_float + 50
            
            # Clip values to valid range [0, 255]
            self.frames[i] = np.clip(frame_float, 0, 255).astype(np.uint8)
            
    def add_extended_white_flash(self,
                                frequency: int = 30,
                                flash_duration: int = 10,
                                fade_duration: int = 5,
                                base_intensity: float = 0.7,
                                peak_intensity: float = 1.0,
                                random_intensity: bool = True,
                                pattern: str = 'center_burst') -> None:
        """
        Add sophisticated white flash effects with fade transitions and patterns.
        
        Args:
            frequency (int): Number of frames between flash sequences
            flash_duration (int): Duration of each flash in frames
            fade_duration (int): Duration of fade in/out in frames
            base_intensity (float): Base intensity of the flash (0.0 to 1.0)
            peak_intensity (float): Peak intensity of the flash (0.0 to 1.0)
            random_intensity (bool): Whether to add random variation to intensity
            pattern (str): Flash pattern type ('center_burst', 'horizontal_sweep', 
                          'vertical_sweep', 'corners', 'random')
        """
        def create_center_burst_mask(frame_shape, progress, center_intensity):
            """Create a radial gradient mask from center"""
            height, width = frame_shape[:2]
            center_y, center_x = height // 2, width // 2
            y, x = np.ogrid[:height, :width]
            
            # Calculate distances from center
            distances = np.sqrt((x - center_x)**2 + (y - center_y)**2)
            max_distance = np.sqrt(center_x**2 + center_y**2)
            
            # Create radial gradient
            mask = (1 - distances / max_distance) * center_intensity
            return np.clip(mask, 0, 1)
    
        def create_horizontal_sweep_mask(frame_shape, progress):
            """Create horizontal sweeping mask"""
            height, width = frame_shape[:2]
            mask = np.zeros((height, width))
            sweep_pos = int(width * progress)
            sweep_width = width // 4
            
            for x in range(max(0, sweep_pos - sweep_width), min(width, sweep_pos + sweep_width)):
                intensity = 1 - abs(x - sweep_pos) / sweep_width
                mask[:, x] = intensity
            return mask
    
        def create_vertical_sweep_mask(frame_shape, progress):
            """Create vertical sweeping mask"""
            height, width = frame_shape[:2]
            mask = np.zeros((height, width))
            sweep_pos = int(height * progress)
            sweep_width = height // 4
            
            for y in range(max(0, sweep_pos - sweep_width), min(height, sweep_pos + sweep_width)):
                intensity = 1 - abs(y - sweep_pos) / sweep_width
                mask[y, :] = intensity
            return mask
    
        def create_corners_mask(frame_shape, progress):
            """Create corner-based flash pattern"""
            height, width = frame_shape[:2]
            mask = np.zeros((height, width))
            corner_size = int(min(height, width) * 0.3 * progress)
            
            # Top-left corner
            mask[:corner_size, :corner_size] = 1
            # Top-right corner
            mask[:corner_size, -corner_size:] = 1
            # Bottom-left corner
            mask[-corner_size:, :corner_size] = 1
            # Bottom-right corner
            mask[-corner_size:, -corner_size:] = 1
            return mask
    
        def create_random_mask(frame_shape, seed=None):
            """Create random pattern mask"""
            if seed is not None:
                np.random.seed(seed)
            return np.random.uniform(0, 1, frame_shape[:2])
    
        for i in range(len(self.frames)):
            frame = self.frames[i].astype(float)
            
            # Check if we should start a flash sequence
            if i % frequency < flash_duration:
                current_frame_in_flash = i % frequency
                
                # Calculate flash progress (0 to 1)
                if current_frame_in_flash < fade_duration:
                    # Fade in
                    progress = current_frame_in_flash / fade_duration
                elif current_frame_in_flash > flash_duration - fade_duration:
                    # Fade out
                    progress = (flash_duration - current_frame_in_flash) / fade_duration
                else:
                    # Full intensity
                    progress = 1.0
                    
                # Apply pattern mask
                if pattern == 'center_burst':
                    intensity = base_intensity + (peak_intensity - base_intensity) * progress
                    mask = create_center_burst_mask(frame.shape, progress, intensity)
                elif pattern == 'horizontal_sweep':
                    mask = create_horizontal_sweep_mask(frame.shape, progress)
                elif pattern == 'vertical_sweep':
                    mask = create_vertical_sweep_mask(frame.shape, progress)
                elif pattern == 'corners':
                    mask = create_corners_mask(frame.shape, progress)
                else:  # random
                    mask = create_random_mask(frame.shape, seed=i)
                
                # Add random variation if enabled
                if random_intensity:
                    mask *= np.random.uniform(0.8, 1.2)
                
                # Expand mask to 3 channels
                mask_3d = np.stack([mask] * 3, axis=2)
                
                # Calculate flash intensity
                flash_intensity = base_intensity + (peak_intensity - base_intensity) * progress
                
                # Create white flash layer
                white_layer = np.ones_like(frame) * 255
                
                # Blend white flash with original frame
                frame = frame * (1 - mask_3d * flash_intensity) + white_layer * (mask_3d * flash_intensity)
                
                # Add overall brightness boost during flash
                frame = frame * (1 + 0.2 * progress)
            
            # Clip values to valid range
            self.frames[i] = np.clip(frame, 0, 255).astype(np.uint8)
            
    def add_red_flash_effect(self,
                            frequency: int = 5,
                            flash_width: int = None,
                            intensity: float = 2.0,
                            min_opacity: float = 0.3,
                            max_opacity: float = 0.7) -> None:
        """
        Add a red light flashing effect to the video.
        
        Args:
            frequency (int): Number of frames between flashes
            flash_width (int): Width of each flash strip. If None, varies randomly
            intensity (float): Brightness intensity of the red flashes
            min_opacity (float): Minimum opacity of the red flash
            max_opacity (float): Maximum opacity of the red flash
        """
        # Define red color (in BGR format for OpenCV)
        RED = np.array([0, 0, 255])
        
        for i in range(len(self.frames)):
            frame = self.frames[i].copy()
            
            # Determine if this frame should have a flash effect
            if i % frequency == 0:
                # Create multiple flash strips
                height = frame.shape[0]
                width = frame.shape[1]
                
                # Determine number of strips (2-4 strips)
                num_strips = np.random.randint(2, 5)
                
                for strip in range(num_strips):
                    # Random or fixed width for flash strip
                    if flash_width is None:
                        strip_width = np.random.randint(50, width // 2)
                    else:
                        strip_width = flash_width
                        
                    # Random position for flash strip
                    start_x = np.random.randint(0, width - strip_width)
                    
                    # Create red flash mask
                    flash_mask = np.zeros_like(frame)
                    flash_mask[:, start_x:start_x + strip_width] = RED
                    
                    # Create gradient fade effect at the edges
                    gradient_width = strip_width // 4
                    for x in range(gradient_width):
                        # Left edge gradient
                        alpha_left = x / gradient_width
                        flash_mask[:, start_x + x] = RED * alpha_left
                        
                        # Right edge gradient
                        alpha_right = (gradient_width - x) / gradient_width
                        flash_mask[:, start_x + strip_width - x - 1] = RED * alpha_right
                    
                    # Blend flash with original frame using random opacity
                    opacity = np.random.uniform(min_opacity, max_opacity) * intensity
                    frame = cv2.addWeighted(frame, 1, flash_mask, opacity, 0)
                
                # Add overall red tint during flash
                red_tint = np.zeros_like(frame)
                red_tint[:, :] = [0, 0, 50]  # Slight red tint in BGR
                frame = cv2.add(frame, red_tint)
                
                # Add some overall brightness to flashing frames
                frame = cv2.convertScaleAbs(frame, alpha=1.1, beta=20)
            
            self.frames[i] = frame
            
    def add_party_flash_effect(self, 
                             flash_frequency: int = 5,
                             flash_colors: List[Tuple[int, int, int]] = None,
                             flash_width: int = None,
                             intensity: float = 2.0) -> None:
        """
        Add party-style flashing effects to the video.
        
        Args:
            flash_frequency (int): Number of frames between flashes
            flash_colors (List[Tuple[int, int, int]]): List of BGR colors for flashes
                                                      Defaults to party colors if None
            flash_width (int): Width of each flash strip. If None, varies randomly
            intensity (float): Brightness intensity of the flashes
        """
        if flash_colors is None:
            # Default party colors (in BGR format)
            flash_colors = [
                (0, 0, 255),    # Red
                (255, 0, 255),  # Magenta
                (255, 255, 0),  # Cyan
                (0, 255, 0),    # Green
                (255, 0, 0),    # Blue
                (0, 255, 255)   # Yellow
            ]
            
        for i in range(len(self.frames)):
            frame = self.frames[i].copy()
            
            # Determine if this frame should have a flash effect
            if i % flash_frequency == 0:
                # Choose random color for this flash
                color = flash_colors[np.random.randint(0, len(flash_colors))]
                
                # Create multiple flash strips
                height = frame.shape[0]
                width = frame.shape[1]
                
                # Determine number of strips (2-5 strips)
                num_strips = np.random.randint(2, 6)
                
                for strip in range(num_strips):
                    # Random or fixed width for flash strip
                    if flash_width is None:
                        strip_width = np.random.randint(50, width // 2)
                    else:
                        strip_width = flash_width
                        
                    # Random position for flash strip
                    start_x = np.random.randint(0, width - strip_width)
                    
                    # Create flash mask
                    flash_mask = np.zeros_like(frame)
                    flash_mask[:, start_x:start_x + strip_width] = color
                    
                    # Blend flash with original frame
                    alpha = np.random.uniform(0.3, 0.7) * intensity
                    frame = cv2.addWeighted(frame, 1, flash_mask, alpha, 0)
                
                # Add some overall brightness to flashing frames
                frame = cv2.convertScaleAbs(frame, alpha=1.2, beta=30)
                
            self.frames[i] = frame
            
    def add_party_flash_by_fps(self,
                              flashes_per_second: float = 2.0,
                              flash_colors: List[Tuple[int, int, int]] = None,
                              flash_width: int = None,
                              intensity: float = 2.0) -> None:
        """
        Add party flash effect specified by flashes per second instead of frame frequency.
        
        Args:
            flashes_per_second (float): Number of flashes per second
            flash_colors (List[Tuple[int, int, int]]): List of BGR colors for flashes
            flash_width (int): Width of each flash strip
            intensity (float): Brightness intensity of the flashes
        """
        # Convert flashes per second to frame frequency
        flash_frequency = max(1, int(self.fps / flashes_per_second))
        self.add_party_flash_effect(
            flash_frequency=flash_frequency,
            flash_colors=flash_colors,
            flash_width=flash_width,
            intensity=intensity
        )

    def add_random_effect_pipeline(self, seed: int = None) -> None:
        """
        Apply a random combination of effects with randomized parameters.
        Creates varied but controlled lighting effects.
        
        Args:
            seed (int, optional): Random seed for reproducibility
        """
        if seed is not None:
            np.random.seed(seed)
            
        # Define ranges for random parameters that create noticeable but not extreme effects
        param_ranges = {
            'brightness_alpha': (1.1, 1.8),    # Brightness multiplier
            'brightness_beta': (10, 40),       # Brightness offset
            'flash_frequency': (3, 8),         # Frames between flashes
            'flash_intensity': (1.3, 2.2),     # Flash intensity
            'block_size': (16, 32),            # Block size for similarity
            'similarity_threshold': (20, 40),   # Threshold for similar pixels
            'transition_speed': (0.8, 1.5)     # Color transition speed
        }
        
        # First, apply random brightness adjustment
        alpha = np.random.uniform(*param_ranges['brightness_alpha'])
        beta = np.random.uniform(*param_ranges['brightness_beta'])
        self.adjust_brightness(alpha=alpha, beta=beta)
        
        # Randomly select number of effects to apply (1-3)
        n_effects = np.random.randint(1, 4)
        
        # List of available effects with their probability weights
        # List of available effects with their probability weights
        effects = [
            (self.add_party_flash_effect, 0.15),  # UPDATED: Changed weight from 0.2 to 0.15
            (self.add_strobe_flash_effect, 0.15), # UPDATED: Changed weight from 0.15 to 0.15 (unchanged)
            (self.add_pixel_intensity_flash, 0.15), # UPDATED: Changed weight from 0.2 to 0.15
            (self.add_similar_pixels_flash, 0.15),  # UPDATED: Added K-means clustering effect
            (self.add_similar_pixels_flash_v2, 0.1),  # UPDATED: Added block-based similarity effect
            (self.add_police_flash_effect, 0.15), # UPDATED: Changed weight from 0.15 to 0.15 (unchanged)
            (self.add_rgb_flash_effect, 0.15) # UPDATED: Changed weight from 0.15 to 0.15 (unchanged)
        ]
        
        # Randomly select and apply effects
        selected_effects = np.random.choice(
            [e[0] for e in effects],
            size=n_effects,
            p=[e[1] for e in effects],
            replace=False
        )
        
        for effect in selected_effects:
            if effect == self.add_party_flash_effect:
                effect(
                    flash_frequency=np.random.randint(*param_ranges['flash_frequency']),
                    flash_width=None,  # Random width
                    intensity=np.random.uniform(*param_ranges['flash_intensity'])
                )
                
            elif effect == self.add_strobe_flash_effect:
                effect(
                    frequency=np.random.randint(*param_ranges['flash_frequency']),
                    intensity=np.random.uniform(*param_ranges['flash_intensity'])
                )
                
            elif effect == self.add_pixel_intensity_flash:
                effect(
                    frequency=np.random.randint(*param_ranges['flash_frequency']),
                    min_intensity=1.1,
                    max_intensity=np.random.uniform(*param_ranges['flash_intensity']),
                    boost_shadows=np.random.choice([True, False])
                )
                
            elif effect == self.add_similar_pixels_flash:
                effect(
                    frequency=np.random.randint(*param_ranges['flash_frequency']),
                    n_clusters=np.random.randint(3, 7),  # Random number of clusters between 3-6
                    flash_intensity=np.random.uniform(*param_ranges['flash_intensity']),
                    similarity_threshold=np.random.uniform(*param_ranges['similarity_threshold'])
                )
                
            elif effect == self.add_police_flash_effect:
                effect(
                    frequency=np.random.randint(*param_ranges['flash_frequency']),
                    block_size=np.random.randint(*param_ranges['block_size']),
                    similarity_threshold=np.random.uniform(*param_ranges['similarity_threshold']),
                    transition_speed=np.random.uniform(*param_ranges['transition_speed'])
                )
                
            elif effect == self.add_rgb_flash_effect:
                effect(
                    frequency=np.random.randint(*param_ranges['flash_frequency']),
                    block_size=np.random.randint(*param_ranges['block_size']),
                    similarity_threshold=np.random.uniform(*param_ranges['similarity_threshold']),
                    transition_speed=np.random.uniform(*param_ranges['transition_speed'])
                )
            elif effect == self.add_similar_pixels_flash:  # UPDATED: Added this entire block
                effect(
                    frequency=np.random.randint(*param_ranges['flash_frequency']),
                    n_clusters=np.random.randint(3, 7),  # Random number of clusters between 3-6
                    flash_intensity=np.random.uniform(*param_ranges['flash_intensity']),
                    similarity_threshold=np.random.uniform(*param_ranges['similarity_threshold'])
                )
            
            elif effect == self.add_similar_pixels_flash_v2:  # UPDATED: Added this entire block
                effect(
                    frequency=np.random.randint(*param_ranges['flash_frequency']),
                    block_size=np.random.randint(*param_ranges['block_size']),
                    flash_intensity=np.random.uniform(*param_ranges['flash_intensity']),
                    similarity_threshold=np.random.uniform(*param_ranges['similarity_threshold'])
                )

    def add_rgb_flash_effect(self,
                          frequency: int = 5,
                          block_size: int = 16,
                          similarity_threshold: float = 30.0,
                          transition_speed: float = 1.0) -> None:
        """
        Create RGB cycling flash effect for similar pixel groups.
        
        Args:
            frequency (int): Number of frames in complete RGB cycle
            block_size (int): Size of blocks to analyze for similarity
            similarity_threshold (float): Maximum color difference to consider pixels similar
            transition_speed (float): Speed of transition between colors (1.0 = normal)
        """
        # Define RGB colors (in BGR format for OpenCV)
        RED = np.array([0, 0, 255])      # Red
        GREEN = np.array([0, 255, 0])    # Green
        BLUE = np.array([255, 0, 0])     # Blue
        
        for i in range(len(self.frames)):
            frame = self.frames[i].astype(float)
            height, width = frame.shape[:2]
            
            # Create similarity mask
            similarity_mask = np.zeros((height, width))
            
            # Process image in blocks
            for y in range(0, height - block_size + 1, block_size):
                for x in range(0, width - block_size + 1, block_size):
                    # Extract block
                    block = frame[y:y + block_size, x:x + block_size]
                    
                    # Calculate mean color of block
                    mean_color = np.mean(block, axis=(0, 1))
                    
                    # Calculate color differences within block
                    diff = np.sqrt(np.sum((block - mean_color) ** 2, axis=2))
                    
                    # Create similarity mask for this block
                    similar_mask = (diff < similarity_threshold)
                    similarity_mask[y:y + block_size, x:x + block_size] = similar_mask
            
            # Calculate color phases using sine waves for smooth transition
            phase = (i % frequency) / frequency * 2 * np.pi * transition_speed
            
            # Create three phases offset by 2π/3 for RGB cycle
            red_intensity = (np.sin(phase) + 1) / 2
            green_intensity = (np.sin(phase + 2 * np.pi / 3) + 1) / 2
            blue_intensity = (np.sin(phase + 4 * np.pi / 3) + 1) / 2
            
            # Create 3D similarity mask
            similarity_mask_3d = np.stack([similarity_mask] * 3, axis=2)
            
            # Mix colors with intensities
            rgb_frame = np.zeros_like(frame)
            
            # Apply each color with its intensity
            rgb_frame += similarity_mask_3d * (RED * red_intensity)
            rgb_frame += similarity_mask_3d * (GREEN * green_intensity)
            rgb_frame += similarity_mask_3d * (BLUE * blue_intensity)
            
            # Blend with original frame where pixels are not similar
            blend_mask = 1 - similarity_mask_3d
            rgb_frame += frame * blend_mask
            
            # Add intensity boost to flashing regions
            max_intensity = max(red_intensity, green_intensity, blue_intensity)
            boost = 40 * max_intensity
            rgb_frame += similarity_mask_3d * boost
            
            # Add color saturation boost for more vibrant colors
            rgb_frame *= 1.2
            
            # Clip values to valid range
            self.frames[i] = np.clip(rgb_frame, 0, 255).astype(np.uint8)

    def add_police_flash_effect(self,
                             frequency: int = 5,
                             block_size: int = 16,
                             similarity_threshold: float = 30.0,
                             transition_speed: float = 1.0) -> None:
        """
        Create police-style red and blue flashing effect for similar pixel groups.
        
        Args:
            frequency (int): Number of frames between color switches
            block_size (int): Size of blocks to analyze for similarity
            similarity_threshold (float): Maximum color difference to consider pixels similar
            transition_speed (float): Speed of transition between colors (1.0 = normal)
        """
        # Define police colors (in BGR format)
        BLUE = np.array([255, 0, 0])
        RED = np.array([0, 0, 255])
        
        for i in range(len(self.frames)):
            frame = self.frames[i].astype(float)
            height, width = frame.shape[:2]
            
            # Create similarity mask
            similarity_mask = np.zeros((height, width))
            
            # Process image in blocks
            for y in range(0, height - block_size + 1, block_size):
                for x in range(0, width - block_size + 1, block_size):
                    # Extract block
                    block = frame[y:y + block_size, x:x + block_size]
                    
                    # Calculate mean color of block
                    mean_color = np.mean(block, axis=(0, 1))
                    
                    # Calculate color differences within block
                    diff = np.sqrt(np.sum((block - mean_color) ** 2, axis=2))
                    
                    # Create similarity mask for this block
                    similar_mask = (diff < similarity_threshold)
                    similarity_mask[y:y + block_size, x:x + block_size] = similar_mask
            
            # Calculate color phase using sine waves for smooth transition
            phase = (i % frequency) / frequency * 2 * np.pi * transition_speed
            red_intensity = (np.sin(phase) + 1) / 2
            blue_intensity = (np.sin(phase + np.pi) + 1) / 2  # Offset by π for alternation
            
            # Create color masks for red and blue
            similarity_mask_3d = np.stack([similarity_mask] * 3, axis=2)
            
            # Mix original frame with police colors based on similarity mask
            police_frame = np.zeros_like(frame)
            
            # Apply red color
            red_component = RED * red_intensity
            police_frame += similarity_mask_3d * red_component
            
            # Apply blue color
            blue_component = BLUE * blue_intensity
            police_frame += similarity_mask_3d * blue_component
            
            # Blend with original frame where pixels are not similar
            blend_mask = 1 - similarity_mask_3d
            police_frame += frame * blend_mask
            
            # Add intensity boost to flashing regions
            boost = 40 * (max(red_intensity, blue_intensity))
            police_frame += similarity_mask_3d * boost
            
            # Clip values to valid range
            self.frames[i] = np.clip(police_frame, 0, 255).astype(np.uint8)

    def add_pixel_intensity_flash(self,
                                frequency: int = 5,
                                min_intensity: float = 1.2,
                                max_intensity: float = 2.5,
                                boost_shadows: bool = True) -> None:
        """
        Add flashing effect where pixels flash relative to their original intensity.
        Brighter pixels will flash more intensely than darker ones.
        
        Args:
            frequency (int): Number of frames between flash pulses
            min_intensity (float): Minimum flash intensity multiplier
            max_intensity (float): Maximum flash intensity multiplier
            boost_shadows (bool): If True, also brightens darker areas slightly
        """
        for i in range(len(self.frames)):
            frame = self.frames[i].astype(float)
            
            # Calculate flash intensity for this frame using sine wave
            # This creates a smooth pulsing effect
            phase = (i % frequency) / frequency * 2 * np.pi
            flash_strength = (np.sin(phase) + 1) / 2  # Normalized to 0-1
            
            # Calculate intensity multiplier for this frame
            current_intensity = min_intensity + (max_intensity - min_intensity) * flash_strength
            
            # Convert frame to grayscale to get luminance values
            gray = cv2.cvtColor(frame.astype(np.uint8), cv2.COLOR_BGR2GRAY).astype(float)
            
            # Normalize luminance values to 0-1
            normalized_luminance = gray / 255.0
            
            # Create intensity mask based on pixel values
            intensity_mask = normalized_luminance * current_intensity
            
            if boost_shadows:
                # Add small constant to boost darker areas
                intensity_mask += 0.2 * flash_strength
            
            # Apply intensity mask to each color channel
            for c in range(3):  # BGR channels
                frame[:, :, c] *= intensity_mask
                
            # Add extra brightness during peak flash moments
            if flash_strength > 0.7:
                frame += 30 * flash_strength
                
            # Clip values to valid range
            self.frames[i] = np.clip(frame, 0, 255).astype(np.uint8)

    def add_similar_pixels_flash(self,
                               frequency: int = 5,
                               n_clusters: int = 5,
                               flash_intensity: float = 2.0,
                               similarity_threshold: float = 30.0) -> None:
        """
        Add flash effect to groups of similar pixels while leaving dissimilar pixels unchanged.
        Uses K-means clustering to identify similar color regions and applies synchronized flashing.
        
        Args:
            frequency (int): Number of frames between flash pulses
            n_clusters (int): Number of color clusters to identify
            flash_intensity (float): Intensity of the flash effect
            similarity_threshold (float): Maximum color difference to consider pixels similar
        """
        
        for i in range(len(self.frames)):
            frame = self.frames[i].astype(float)
            
            # Reshape frame for clustering
            height, width = frame.shape[:2]
            pixels = frame.reshape(-1, 3)
            
            # Perform color clustering
            kmeans = KMeans(n_clusters=n_clusters, n_init=1)
            labels = kmeans.fit_predict(pixels)
            centers = kmeans.cluster_centers_
            
            # Calculate flash intensity for this frame using sine wave
            phase = (i % frequency) / frequency * 2 * np.pi
            flash_strength = (np.sin(phase) + 1) / 2
            
            # Create mask for similar pixels
            flash_mask = np.zeros((height * width))
            
            # For each cluster
            for cluster_idx in range(n_clusters):
                cluster_pixels = pixels[labels == cluster_idx]
                cluster_center = centers[cluster_idx]
                
                # Calculate color distances within cluster
                distances = np.sqrt(np.sum((cluster_pixels - cluster_center) ** 2, axis=1))
                
                # Identify pixels similar enough to cluster center
                similar_pixels = distances < similarity_threshold
                
                # Apply flash to similar pixels in this cluster
                cluster_mask = (labels == cluster_idx)
                flash_mask[cluster_mask] = similar_pixels * flash_strength
            
            # Reshape mask back to image dimensions and expand to 3 channels
            flash_mask = flash_mask.reshape(height, width)
            flash_mask = np.stack([flash_mask] * 3, axis=2)
            
            # Apply flash effect
            flash_factor = 1.0 + (flash_intensity - 1.0) * flash_mask
            frame *= flash_factor
            
            # Add extra brightness to flashing regions during peaks
            if flash_strength > 0.6:
                frame += flash_mask * 40 * flash_strength
            
            # Clip values to valid range
            self.frames[i] = np.clip(frame, 0, 255).astype(np.uint8)

    def add_similar_pixels_flash_v2(self,
                                  frequency: int = 5,
                                  block_size: int = 16,
                                  flash_intensity: float = 2.0,
                                  similarity_threshold: float = 30.0) -> None:
        """
        Alternative version that processes image in blocks for more localized similarity detection.
        Uses block-wise color statistics to identify and flash similar regions.
        
        Args:
            frequency (int): Number of frames between flash pulses
            block_size (int): Size of blocks to analyze for similarity
            flash_intensity (float): Intensity of the flash effect
            similarity_threshold (float): Maximum color difference to consider pixels similar
        """
        for i in range(len(self.frames)):
            frame = self.frames[i].astype(float)
            height, width = frame.shape[:2]
            
            # Create flash mask
            flash_mask = np.zeros((height, width))
            
            # Calculate flash intensity using sine wave
            phase = (i % frequency) / frequency * 2 * np.pi
            flash_strength = (np.sin(phase) + 1) / 2
            
            # Process image in blocks
            for y in range(0, height - block_size + 1, block_size):
                for x in range(0, width - block_size + 1, block_size):
                    # Extract block
                    block = frame[y:y + block_size, x:x + block_size]
                    
                    # Calculate mean color of block
                    mean_color = np.mean(block, axis=(0, 1))
                    
                    # Calculate color differences within block
                    diff = np.sqrt(np.sum((block - mean_color) ** 2, axis=2))
                    
                    # Create similarity mask for this block
                    similar_mask = (diff < similarity_threshold)
                    flash_mask[y:y + block_size, x:x + block_size] = similar_mask * flash_strength
            
            # Expand mask to 3 channels
            flash_mask = np.stack([flash_mask] * 3, axis=2)
            
            # Apply flash effect
            flash_factor = 1.0 + (flash_intensity - 1.0) * flash_mask
            frame *= flash_factor
            
            # Add extra brightness to flashing regions during peaks
            if flash_strength > 0.6:
                frame += flash_mask * 40 * flash_strength
            
            # Clip values to valid range
            self.frames[i] = np.clip(frame, 0, 255).astype(np.uint8)

    def add_strobe_flash_effect(self, 
                              frequency: int = 3,
                              intensity: float = 1.5) -> None:
        """
        Add a strobing flash effect that affects the entire frame.
        
        Args:
            frequency (int): Number of frames between flashes
            intensity (float): Brightness intensity of the strobe effect
        """
        for i in range(len(self.frames)):
            if i % frequency == 0:
                # Create bright flash
                frame_float = self.frames[i].astype(float)
                frame_float *= intensity
                frame_float += 80
                self.frames[i] = np.clip(frame_float, 0, 255).astype(np.uint8)
            else:
                # Slightly darken non-flash frames for contrast
                self.frames[i] = cv2.convertScaleAbs(self.frames[i], alpha=0.8, beta=0)
                
    def add_combined_flash_pipeline(self,
                                  seed: Optional[int] = None,
                                  intensity_range: Tuple[float, float] = (1.3, 2.2),
                                  frequency_range: Tuple[int, int] = (3, 8)) -> None:
        """
        Apply a coordinated pipeline of white flash, red flash, and extended white flash effects.
        
        Args:
            seed (int, optional): Random seed for reproducible results
            intensity_range (tuple): Range of intensity values (min, max)
            frequency_range (tuple): Range of frequency values (min, max)
        """
        if seed is not None:
            np.random.seed(seed)
            
        # Define ranges for random parameters
        param_ranges = {
            'base_intensity': (intensity_range[0], intensity_range[1] - 0.2),
            'peak_intensity': (intensity_range[1] - 0.2, intensity_range[1]),
            'frequency': frequency_range,
            'flash_duration': (5, 15),
            'fade_duration': (3, 7)
        }
        
        # List of available flash patterns for extended white flash
        flash_patterns = ['center_burst', 'horizontal_sweep', 'vertical_sweep', 'corners']
        
        # Step 1: Apply basic strobe effect with lower intensity
        base_frequency = np.random.randint(*param_ranges['frequency'])
        self.add_strobe_flash_effect(
            frequency=base_frequency,
            intensity=np.random.uniform(*param_ranges['base_intensity'])
        )
        
        # Step 2: Add red flash effect
        red_frequency = max(5, base_frequency + np.random.randint(-2, 3))  # Vary slightly from base
        self.add_red_flash_effect(
            frequency=red_frequency,
            flash_width=None,  # Random width
            intensity=np.random.uniform(*param_ranges['base_intensity']),
            min_opacity=0.3,
            max_opacity=0.7
        )
        
        # Step 3: Add extended white flash with random pattern
        extended_frequency = base_frequency * 2  # Less frequent than base flashes
        selected_pattern = np.random.choice(flash_patterns)
        
        self.add_extended_white_flash(
            frequency=extended_frequency,
            flash_duration=np.random.randint(*param_ranges['flash_duration']),
            fade_duration=np.random.randint(*param_ranges['fade_duration']),
            base_intensity=np.random.uniform(*param_ranges['base_intensity']),
            peak_intensity=np.random.uniform(*param_ranges['peak_intensity']),
            random_intensity=True,
            pattern=selected_pattern
        )
    
    def add_flash_sequence(self,
                          sequence_type: str = 'random',
                          duration_seconds: float = 5.0,
                          intensity: float = 1.5) -> None:
        """
        Apply a predefined sequence of flash effects.
        
        Args:
            sequence_type (str): Type of sequence ('random', 'alternating', 'crescendo', 'decrescendo')
            duration_seconds (float): Duration of the sequence in seconds
            intensity (float): Base intensity multiplier for the effects
        """
        # Calculate number of frames for the sequence
        sequence_frames = int(duration_seconds * self.fps)
        current_frame = 0
        
        if sequence_type == 'random':
            # Random sequence of effects
            while current_frame < len(self.frames):
                effect = np.random.choice(['white', 'red', 'extended'])
                duration = np.random.randint(1, 3) * self.fps  # 1-2 seconds
                
                if effect == 'white':
                    self.add_strobe_flash_effect(
                        frequency=3,
                        intensity=intensity
                    )
                elif effect == 'red':
                    self.add_red_flash_effect(
                        frequency=5,
                        intensity=intensity
                    )
                else:  # extended
                    self.add_extended_white_flash(
                        frequency=30,
                        flash_duration=10,
                        base_intensity=intensity * 0.7,
                        peak_intensity=intensity
                    )
                    
                current_frame += duration
                
        elif sequence_type == 'alternating':
            # Alternating between red and white flashes
            is_red = True
            while current_frame < len(self.frames):
                if is_red:
                    self.add_red_flash_effect(
                        frequency=4,
                        intensity=intensity,
                        flash_width=None
                    )
                else:
                    self.add_strobe_flash_effect(
                        frequency=4,
                        intensity=intensity
                    )
                
                current_frame += self.fps  # Switch every second
                is_red = not is_red
                
        elif sequence_type == 'crescendo':
            # Gradually increasing intensity
            steps = 5
            for i in range(steps):
                step_intensity = intensity * (0.5 + 0.5 * (i + 1) / steps)
                
                self.add_combined_flash_pipeline(
                    intensity_range=(step_intensity * 0.8, step_intensity),
                    frequency_range=(max(2, 8 - i), max(3, 10 - i))
                )
                
        elif sequence_type == 'decrescendo':
            # Gradually decreasing intensity
            steps = 5
            for i in range(steps):
                step_intensity = intensity * (1.0 - 0.5 * i / steps)
                
                self.add_combined_flash_pipeline(
                    intensity_range=(step_intensity * 0.8, step_intensity),
                    frequency_range=(3 + i, 5 + i)
                )
        
    def presets_flash_effects(self, preset_name: str = 'party') -> None:
        """
        Apply predefined combinations of flash effects.
        
        Args:
            preset_name (str): Name of the preset ('party', 'emergency', 'subtle', 'intense')
        """
        if preset_name == 'party':
            self.add_combined_flash_pipeline(
                intensity_range=(1.3, 1.8),
                frequency_range=(4, 7)
            )
            
        elif preset_name == 'emergency':
            # Emergency-style alternating red and white flashes
            self.add_flash_sequence(
                sequence_type='alternating',
                duration_seconds=float('inf'),  # Entire video
                intensity=1.7
            )
            
        elif preset_name == 'subtle':
            # Subtle flashing effects
            self.add_combined_flash_pipeline(
                intensity_range=(1.1, 1.4),
                frequency_range=(8, 12)
            )
            
        elif preset_name == 'intense':
            # Intense flashing with crescendo
            self.add_flash_sequence(
                sequence_type='crescendo',
                duration_seconds=float('inf'),
                intensity=2.0
            )
    def add_fullscreen_red_flash(self,
                                frequency: int = 5,
                                intensity: float = 1.5,
                                fade_duration: int = 3,
                                min_opacity: float = 0.4,
                                max_opacity: float = 0.8) -> None:
        """
        Add a full-screen red flash effect.
        
        Args:
            frequency (int): Number of frames between flashes
            intensity (float): Intensity of the red flash
            fade_duration (int): Number of frames for fade in/out
            min_opacity (float): Minimum opacity of the red overlay
            max_opacity (float): Maximum opacity of the red overlay
        """
        # Define pure red color (in BGR format for OpenCV)
        RED = np.array([0, 0, 255])
        
        for i in range(len(self.frames)):
            frame = self.frames[i].astype(float)
            
            # Check if this is a flash frame
            if i % frequency < fade_duration * 2:
                # Calculate flash progress
                if i % frequency < fade_duration:
                    # Fade in
                    progress = (i % frequency) / fade_duration
                else:
                    # Fade out
                    progress = 1 - ((i % frequency - fade_duration) / fade_duration)
                
                # Create red overlay
                red_overlay = np.full_like(frame, RED)
                
                # Calculate current opacity based on progress
                current_opacity = min_opacity + (max_opacity - min_opacity) * progress
                current_opacity *= intensity
                
                # Blend red overlay with original frame
                frame = cv2.addWeighted(
                    frame,
                    1.0,
                    red_overlay,
                    current_opacity,
                    0
                )
                
                # Add extra brightness during flash
                frame = cv2.convertScaleAbs(
                    frame.astype(np.uint8),
                    alpha=1.0 + (0.2 * progress),
                    beta=20 * progress
                ).astype(float)
            
            # Ensure values stay within valid range
            self.frames[i] = np.clip(frame, 0, 255).astype(np.uint8)

    def add_random_partition_effects_pipeline(self, 
                                             output_folder: str = "/corrupted_videos",
                                             min_segment_duration: float = 1.0,
                                             seed: Optional[int] = None) -> str:
        """
        Generate a corrupted video by randomly partitioning it into segments and
        applying random effects to each segment.
        
        Args:
            output_folder (str): Folder to save the corrupted video
            min_segment_duration (float): Minimum segment duration in seconds
            seed (Optional[int]): Random seed for reproducibility
            
        Returns:
            str: Path to the output video
        """
        if seed is not None:
            random.seed(seed)
            np.random.seed(seed)
        
        # Get video properties
        video_info = self.get_video_info()
        video_duration = video_info['duration']
        fps = video_info['fps']
        total_frames = video_info['frame_count']
        
        # Create output folder if it doesn't exist
        os.makedirs(output_folder, exist_ok=True)
        
        # Generate output filename
        input_filename = Path(self.video_path).stem
        output_filepath = os.path.join(output_folder, f"{input_filename}.mp4")

        # Calculate minimum segment length in frames
        min_segment_frames = int(min_segment_duration * fps)
        
        # Create a copy of the original frames
        original_frames = self.frames.copy()
        
        # UPDATED: Better way to calculate maximum number of segments
        # Ensure we don't try to create more segments than possible with the min_segment_duration
        max_possible_segments = total_frames // min_segment_frames
        max_segments = min(int(video_duration / min_segment_duration), max_possible_segments)
        
        # Limit to a reasonable number to avoid too many tiny segments
        max_segments = min(max_segments, 10)  # UPDATED: Added a hard cap on segments
        
        # Choose a random number of segments (between 1 and max_segments)
        num_segments = random.randint(1, max_segments)
        
        # UPDATED: Completely revised partition generation to ensure minimum segment size
        partition_frames = []
        
        if num_segments > 1:
            # Calculate usable frame range (excluding boundaries)
            usable_start = min_segment_frames
            usable_end = total_frames - min_segment_frames
            usable_range = usable_end - usable_start
            
            # Check if we have enough frames for the requested segments
            if usable_range < (num_segments - 1) * min_segment_frames:
                # Adjust number of segments if not enough frames
                num_segments = (usable_range // min_segment_frames) + 1
                print(f"Adjusted number of segments to {num_segments} due to minimum segment duration constraint")
            
            if num_segments > 1:
                # UPDATED: Use a different approach for partition generation
                # Create partitions ensuring each segment is at least min_segment_frames in length
                
                # Calculate how many internal partition points we need
                n_partitions = num_segments - 1
                
                # Create a list of potential partition points
                possible_partitions = []
                
                # The first partition can be anywhere from min_segment_frames to 
                # total_frames - (n_partitions * min_segment_frames)
                first_partition_max = total_frames - (n_partitions * min_segment_frames)
                possible_partitions.append(random.randint(min_segment_frames, first_partition_max))
                
                # For each subsequent partition, ensure it's at least min_segment_frames
                # away from the previous one
                for i in range(1, n_partitions):
                    min_pos = possible_partitions[-1] + min_segment_frames
                    max_pos = total_frames - ((n_partitions - i) * min_segment_frames)
                    
                    if min_pos >= max_pos:
                        # Cannot satisfy constraints, reduce number of segments
                        num_segments = i + 1
                        print(f"Further adjusted number of segments to {num_segments} due to constraints")
                        break
                    
                    possible_partitions.append(random.randint(min_pos, max_pos))
                
                partition_frames = sorted(possible_partitions)
        
        # Add the beginning and end points
        segment_boundaries = [0] + partition_frames + [total_frames]
        
        print(f"Partitioning video into {num_segments} segments")
        
        # Verify segments are of minimum size (debug)
        for i in range(len(segment_boundaries) - 1):
            segment_size = segment_boundaries[i+1] - segment_boundaries[i]
            segment_duration = segment_size / fps
            print(f"Segment {i+1}: frames {segment_boundaries[i]}-{segment_boundaries[i+1]} ({segment_duration:.2f} seconds)")
            if segment_size < min_segment_frames:
                print(f"WARNING: Segment {i+1} is smaller than minimum size ({segment_size} < {min_segment_frames})")
    
        
        
        # List of effects to choose from (exclude K-means based effects)
        effects = [
            self.add_red_flash_effect,
            self.add_party_flash_effect,
            self.add_extended_white_flash,
            self.add_fullscreen_red_flash,
            self.add_pixel_intensity_flash,
            self.add_police_flash_effect,
            self.add_rgb_flash_effect,
            self.add_similar_pixels_flash,     # UPDATED: Added K-means based effect
            self.add_similar_pixels_flash_v2,  # UPDATED: Added block-based similarity effect
        ]
        
        # Effects that take parameters
        parametrized_effects = [
            lambda: self.add_strobe_flash_effect(frequency=random.randint(5, 30), intensity=random.uniform(2.0, 3.5)),
            lambda: self.add_red_flash_effect(frequency=random.randint(5, 30), intensity=random.uniform(2.0, 3.5)),
            lambda: self.add_party_flash_effect(flash_frequency=random.randint(5, 30), intensity=random.uniform(2.0, 3.5)),
            lambda: self.add_extended_white_flash(
                frequency=random.randint(5, 30),
                flash_duration=random.randint(5, 15),
                fade_duration=random.randint(2, 8),
                pattern=random.choice(['center_burst', 'horizontal_sweep', 'vertical_sweep', 'corners'])
            ),
            lambda: self.add_fullscreen_red_flash(
                frequency=random.randint(5, 30),
                intensity=random.uniform(2.0, 3.5)
            ),
            lambda: self.add_pixel_intensity_flash(
                frequency=random.randint(5, 30),
                max_intensity=random.uniform(2.5, 4.0)  # More extreme intensity
            ),
            lambda: self.add_police_flash_effect(
                frequency=random.randint(5, 30),
                block_size=random.choice([8, 16, 32]),
                transition_speed=random.uniform(1.0, 2.5)
            ),
            lambda: self.add_rgb_flash_effect(
                frequency=random.randint(5, 30),
                block_size=random.choice([8, 16, 32]),
                transition_speed=random.uniform(1.0, 2.5)
            ),
             # UPDATED: Added K-means based effects with parameters (these two lambda functions)
            lambda: self.add_similar_pixels_flash(
                frequency=random.randint(5, 30),
                n_clusters=random.randint(3, 7),
                flash_intensity=random.uniform(2.0, 3.5),
                similarity_threshold=random.uniform(20, 50)
            ),
            lambda: self.add_similar_pixels_flash_v2(
                frequency=random.randint(5, 30),
                block_size=random.choice([8, 16, 32]),
                flash_intensity=random.uniform(2.0, 3.5),
                similarity_threshold=random.uniform(20, 50)
            )
        ]
        
        # Combine regular and parametrized effects
        all_effects = effects + parametrized_effects
        
        # Process each segment
        for i in range(len(segment_boundaries) - 1):
            start_frame = segment_boundaries[i]
            end_frame = segment_boundaries[i + 1]
            
            # Calculate segment duration in seconds for logging
            segment_duration = (end_frame - start_frame) / fps
            print(f"Segment {i+1}: frames {start_frame}-{end_frame} ({segment_duration:.2f} seconds)")
            
            # Temporarily set self.frames to just this segment for effect application
            self.frames = original_frames[start_frame:end_frame].copy()
            
            # Choose a random effect for this segment
            # Use the random lambda functions for parametrized effects
            selected_effect = random.choice(parametrized_effects)
            
            # Apply the selected effect
            effect_name = selected_effect.__name__ if hasattr(selected_effect, '__name__') else "parametrized_effect"
            print(f"Applying effect: {effect_name} to segment {i+1}")
            
            try:
                selected_effect()
            except Exception as e:
                print(f"Error applying effect to segment {i+1}: {e}")
                # If effect fails, keep the original segment
                pass
            
            # Store the modified segment back in original_frames
            original_frames[start_frame:end_frame] = self.frames
        
        # Restore all frames with the modified segments
        self.frames = original_frames
        
        # Save the corrupted video
        self.save_video(output_filepath)
        print(f"Corrupted video saved to: {output_filepath}")
        
        return output_filepath


    def add_random_partition_effects_pipeline_v2(self, 
                                             output_folder: str = "/corrupted_videos",
                                             min_segment_duration: float = 1.0,
                                             no_effect_chance: float = 0.38) -> str:
        """
        Generate a corrupted video by randomly partitioning it into segments and
        applying random effects to each segment. Has a chance to apply no effect to some segments.
        All effects have equal probability of being selected.
        
        Args:
            output_folder (str): Folder to save the corrupted video
            min_segment_duration (float): Minimum segment duration in seconds
            no_effect_chance (float): Probability (0.0 to 1.0) of applying no effect to a segment
            
        Returns:
            str: Path to the output video
        """
        
        # Get video properties
        video_info = self.get_video_info()
        video_duration = video_info['duration']
        fps = video_info['fps']
        total_frames = video_info['frame_count']
        
        # Create output folder if it doesn't exist
        os.makedirs(output_folder, exist_ok=True)
        
        # Generate output filename
        input_filename = Path(self.video_path).stem
        output_filepath = os.path.join(output_folder, f"{input_filename}.mp4")
    
        # Calculate minimum segment length in frames
        min_segment_frames = int(min_segment_duration * fps)
        
        # Create a copy of the original frames
        original_frames = self.frames.copy()
        
        # Better way to calculate maximum number of segments
        # Ensure we don't try to create more segments than possible with the min_segment_duration
        max_possible_segments = total_frames // min_segment_frames
        max_segments = min(int(video_duration / min_segment_duration), max_possible_segments)
        
        # Limit to a reasonable number to avoid too many tiny segments
        max_segments = min(max_segments, 10)  # Hard cap on segments
        
        # Choose a random number of segments (between 1 and max_segments)
        num_segments = random.randint(1, max_segments)
        
        # Improved partition generation with more balanced segment distribution
        partition_frames = []
        
        if num_segments > 1:
            # Calculate the "ideal" segment size if all were equal
            ideal_segment_size = total_frames / num_segments
            
            # Calculate available space for randomization per segment
            extra_space_per_segment = ideal_segment_size - min_segment_frames
            
            if extra_space_per_segment < 0:
                # Can't satisfy constraints with current segment count
                num_segments = max(1, total_frames // min_segment_frames)
                print(f"Adjusted number of segments to {num_segments} due to minimum segment duration constraint")
                ideal_segment_size = total_frames / num_segments
                extra_space_per_segment = ideal_segment_size - min_segment_frames
            
            last_point = 0
            partition_frames = []
            
            # Generate partition points
            for i in range(num_segments - 1):
                # Calculate allowed range for this partition
                min_pos = last_point + min_segment_frames
                
                # For more uniform distribution, center the randomness around the ideal point
                ideal_pos = last_point + ideal_segment_size
                
                # Define deviation range (reducing for later segments to ensure we don't run out of space)
                remaining_segments = num_segments - i
                deviation = min(extra_space_per_segment * 0.8, 
                                (total_frames - min_pos - (min_segment_frames * (remaining_segments - 1))) / remaining_segments)
                
                # Calculate range with deviation around ideal position
                min_range = max(min_pos, ideal_pos - deviation)
                max_range = min(total_frames - ((remaining_segments - 1) * min_segment_frames), 
                                ideal_pos + deviation)
                
                if min_range >= max_range:
                    # Partition constraints cannot be satisfied, reduce segment count
                    num_segments = i + 1
                    print(f"Further adjusted number of segments to {num_segments} due to constraints")
                    break
                
                # Create partition point
                next_point = random.randint(int(min_range), int(max_range))
                partition_frames.append(next_point)
                last_point = next_point
            
            # Ensure partition points are sorted
            partition_frames = sorted(partition_frames)
        
        # Add the beginning and end points
        segment_boundaries = [0] + partition_frames + [total_frames]
        
        print(f"Partitioning video into {num_segments} segments")
        
        # Verify segments are of minimum size (debug)
        for i in range(len(segment_boundaries) - 1):
            segment_size = segment_boundaries[i+1] - segment_boundaries[i]
            segment_duration = segment_size / fps
            print(f"Segment {i+1}: frames {segment_boundaries[i]}-{segment_boundaries[i+1]} ({segment_duration:.2f} seconds)")
            if segment_size < min_segment_frames:
                print(f"WARNING: Segment {i+1} is smaller than minimum size ({segment_size} < {min_segment_frames})")
        
        # Define all effects with direct method references and parameters
        # No weights - all effects have equal probability
        available_effects = [
            {"effect": (self.add_red_flash_effect, [{"frequency": random.randint(1, 6), "intensity": random.uniform(2.0, 3.5)}]), 
             "name": "add_red_flash_effect"},
            {"effect": (self.add_party_flash_effect, [{"flash_frequency": random.randint(1, 6), "intensity": random.uniform(2.0, 3.5)}]), 
             "name": "add_party_flash_effect"},
            {"effect": (self.add_extended_white_flash, [{"frequency": random.randint(1, 6), 
                                                        "flash_duration": random.randint(5, 15),
                                                        "fade_duration": random.randint(2, 8),
                                                        "pattern": random.choice(['center_burst', 'horizontal_sweep', 'vertical_sweep', 'corners'])}]), 
             "name": "add_extended_white_flash"},
            {"effect": (self.add_fullscreen_red_flash, [{"frequency": random.randint(1, 6), "intensity": random.uniform(2.0, 3.5)}]), 
             "name": "add_fullscreen_red_flash"},
            {"effect": (self.add_pixel_intensity_flash, [{"frequency": random.randint(1, 6), "max_intensity": random.uniform(2.5, 4.0)}]), 
             "name": "add_pixel_intensity_flash"},
            {"effect": (self.add_police_flash_effect, [{"frequency": random.randint(1, 6), 
                                                       "block_size": random.choice([8, 16, 32]),
                                                       "transition_speed": random.uniform(1.0, 2.5)}]), 
             "name": "add_police_flash_effect"},
            {"effect": (self.add_rgb_flash_effect, [{"frequency": random.randint(1, 6), 
                                                    "block_size": random.choice([8, 16, 32]),
                                                    "transition_speed": random.uniform(1.0, 2.5)}]), 
             "name": "add_rgb_flash_effect"},
            {"effect": (self.add_similar_pixels_flash, [{"frequency": random.randint(1, 6),
                                                        "n_clusters": random.randint(3, 7),
                                                        "flash_intensity": random.uniform(2.0, 3.5),
                                                        "similarity_threshold": random.uniform(20, 50)}]), 
             "name": "add_similar_pixels_flash"},
            {"effect": (self.add_similar_pixels_flash_v2, [{"frequency": random.randint(1, 6),
                                                           "block_size": random.choice([8, 16, 32]),
                                                           "flash_intensity": random.uniform(2.0, 3.5),
                                                           "similarity_threshold": random.uniform(20, 50)}]), 
             "name": "add_similar_pixels_flash_v2"}
        ]
        
        # Process each segment
        for i in range(len(segment_boundaries) - 1):
            start_frame = segment_boundaries[i]
            end_frame = segment_boundaries[i + 1]
            
            # Calculate segment duration in seconds for logging
            segment_duration = (end_frame - start_frame) / fps
            print(f"Segment {i+1}: frames {start_frame}-{end_frame} ({segment_duration:.2f} seconds)")
            
            # Temporarily set self.frames to just this segment for effect application
            self.frames = original_frames[start_frame:end_frame].copy()
            
            # Determine if we should apply an effect to this segment (35% chance of no effect)
            apply_effect = random.random() > no_effect_chance
            
            if apply_effect:
                # Choose a random effect (equal probability)
                selected_effect_data = random.choice(available_effects)
                
                # Extract the effect function and its parameters
                effect_func, effect_params = selected_effect_data["effect"]
                effect_name = selected_effect_data["name"]
                
                print(f"Applying effect: {effect_name} to segment {i+1}")
                
                try:
                    # Apply the effect with its parameters
                    if effect_params:
                        # If there are parameters, unpack and call
                        effect_func(**effect_params[0])
                    else:
                        # If no parameters, call directly
                        effect_func()
                except Exception as e:
                    print(f"Error applying effect to segment {i+1}: {e}")
                    # If effect fails, keep the original segment
                    pass
            else:
                print(f"No effect applied to segment {i+1} (random chance)")
            
            # Store the modified segment back in original_frames
            original_frames[start_frame:end_frame] = self.frames
        
        # Restore all frames with the modified segments
        self.frames = original_frames
        
        # Save the corrupted video
        self.save_video(output_filepath)
        print(f"Corrupted video saved to: {output_filepath}")
        
        return output_filepath

In [4]:
def process_videos_in_folder(input_folder="raw_videos_720p", 
                            output_folder="corrupted_videos_720P",
                            file_extensions=('.mp4', '.avi', '.mov', '.mkv'),
                            min_segment_duration=1.0):
    """
    Process all videos in the input folder and apply random effects.
    
    Args:
        input_folder (str): Folder containing raw videos
        output_folder (str): Folder to save corrupted videos
        file_extensions (tuple): Video file extensions to process
        min_segment_duration (float): Minimum segment duration in seconds
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Find all video files in the input folder
    video_files = []
    for ext in file_extensions:
        video_files.extend(list(Path(input_folder).glob(f"*{ext}")))
    
    if not video_files:
        print(f"No video files found in {input_folder} with extensions {file_extensions}")
        return
    
    print(f"Found {len(video_files)} video files to process")
    
    # Process each video
    for i, video_path in enumerate(video_files):
        print(f"\n[{i+1}/{len(video_files)}] Processing: {video_path}")
        start_time = time.time()
        
        try:
            # Create CustomVideo instance
            video = CustomVideo(str(video_path))
            
            # Apply random effects
            output_path = video.add_random_partition_effects_pipeline_v2(
                output_folder=output_folder,
                min_segment_duration=min_segment_duration
            )
            
            # Calculate and display processing time
            processing_time = time.time() - start_time
            print(f"Processing completed in {processing_time:.2f} seconds")
            print(f"Corrupted video saved to: {output_path}")
            
        except Exception as e:
            print(f"Error processing {video_path}: {e}")
            continue
    
    print("\nAll videos processed!")

if __name__ == "__main__":
    # You can customize these parameters
    process_videos_in_folder(
    )

Found 93 video files to process

[1/93] Processing: raw_videos_720p/Lecture_720P-1f22_crf_10_ss_00_t_20.0.mp4
Partitioning video into 5 segments
Segment 1: frames 0-86 (2.87 seconds)
Segment 2: frames 86-145 (1.97 seconds)
Segment 3: frames 145-293 (4.93 seconds)
Segment 4: frames 293-461 (5.60 seconds)
Segment 5: frames 461-599 (4.60 seconds)
Segment 1: frames 0-86 (2.87 seconds)
No effect applied to segment 1 (random chance)
Segment 2: frames 86-145 (1.97 seconds)
Applying effect: add_similar_pixels_flash to segment 2
Segment 3: frames 145-293 (4.93 seconds)
Applying effect: add_fullscreen_red_flash to segment 3
Segment 4: frames 293-461 (5.60 seconds)
No effect applied to segment 4 (random chance)
Segment 5: frames 461-599 (4.60 seconds)
No effect applied to segment 5 (random chance)
Corrupted video saved to: corrupted_videos_720P/Lecture_720P-1f22_crf_10_ss_00_t_20.0.mp4
Processing completed in 23.22 seconds
Corrupted video saved to: corrupted_videos_720P/Lecture_720P-1f22_crf_10_s

/home/kaan-eren/jupyter_env/lib/python3.12/site-packages/sklearn/base.py:1473: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


Corrupted video saved to: corrupted_videos_720P/Animation_720P-06a6_crf_10_ss_00_t_20.0.mp4
Processing completed in 75.54 seconds
Corrupted video saved to: corrupted_videos_720P/Animation_720P-06a6_crf_10_ss_00_t_20.0.mp4

[41/93] Processing: raw_videos_720p/Lecture_720P-3b7f_crf_10_ss_00_t_20.0.mp4
Partitioning video into 9 segments
Segment 1: frames 0-57 (1.90 seconds)
Segment 2: frames 57-101 (1.47 seconds)
Segment 3: frames 101-162 (2.03 seconds)
Segment 4: frames 162-249 (2.90 seconds)
Segment 5: frames 249-314 (2.17 seconds)
Segment 6: frames 314-402 (2.93 seconds)
Segment 7: frames 402-496 (3.13 seconds)
Segment 8: frames 496-548 (1.73 seconds)
Segment 9: frames 548-600 (1.73 seconds)
Segment 1: frames 0-57 (1.90 seconds)
Applying effect: add_extended_white_flash to segment 1
Segment 2: frames 57-101 (1.47 seconds)
No effect applied to segment 2 (random chance)
Segment 3: frames 101-162 (2.03 seconds)
Applying effect: add_extended_white_flash to segment 3
Segment 4: frames 162-2